In [1]:
# Packages to be used
import networkx as nx
from networkx.algorithms import approximation
from networkx.algorithms import community
from networkx.algorithms import centrality
import numpy as np
import matplotlib.pyplot as plt
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
import random
# import scipy
# import math
# import sys
import pandas as pd
# from fractions import Fraction
# import timeit
# from time import sleep
import igraph as ig
import time
import collections
# import heapq as heap
# from tqdm import tqdm 
import community
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend

no display found. Using non-interactive Agg backend


In [2]:
# Load datasets, outside of definitions

# G1 = nx.read_edgelist("Wiki-Vote.txt", create_using=nx.DiGraph(), nodetype= int)
# print(G1)
G2 = nx.read_edgelist("Wiki-Vote.txt", create_using=nx.Graph(), nodetype=int)
print(G2)


Graph with 7115 nodes and 100762 edges


In [ ]:
# Making a directed graph undirected

G1_undirected = G1.to_undirected()
print(G1_undirected)

Heuristics to block/remove nodes

In [3]:
# Centrality measures

# Degree
# sort_degree = sorted(dict(G2.degree).items(), key=lambda item: item[1], reverse=True)
# print(sort_degree)

# Degree centrality

# degree_centrality = nx.centrality.degree_centrality(G2)
# sort_degree_centrality = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)
# print("Done with degree centrality")

# Betweenness

betweenness = nx.betweenness_centrality(G2)
sort_betweenness = sorted(betweenness.items(), key=lambda item: item[1], reverse=True)
print("Done with betweenness centrality")


# Closeness

# closeness = nx.closeness_centrality(G2)
# sort_closeness = sorted(closeness.items(), key=lambda item: item[1], reverse=True)
# print("Done with closeness centrality")


# Influence maximization
# Pagerank

# pagerank = nx.pagerank(G2)
# sort_pagerank = sorted(pagerank.items(), key=lambda item: item[1], reverse=True)
# print("Done with pagerank centrality")

# Eigenvector

# eigenvector = nx.eigenvector_centrality(G2, max_iter=300)
# sort_eigenvector = sorted(eigenvector.items(), key=lambda item: item[1], reverse=True)
# print("Done with eigenvector centrality")

Done with betweenness centrality


In [ ]:
# To only get the keys from the sorted list

only_keys = [k for k,v in sort_degree_centrality]
print(only_keys)

# Take the top 50 nodes to be used as seeds later
first_100 = only_keys[0:100]
print(first_100)

In [ ]:
# Add weight to the edges of the graph. To be used as thresholds for the ICM

degree_dict = dict(G2.degree())
weights = []
for u, v, data in G2.edges(data=True):
    ratio = 1 / degree_dict[v]
    data['weight'] = ratio
    weights.append(ratio)

print(weights)

In [ ]:
# Create a copy of the original graph to be modified later, by removing nodes
G2_copy = G2.copy()


G2_copy.remove_nodes_from(first_100)
print(G2_copy)

The following is the centrality measures for the new graph (After removing the nodes)

In [ ]:
# Centrality measures

# Degree
# sort_degree = sorted(dict(G2.degree).items(), key=lambda item: item[1], reverse=True)
# print(sort_degree)

# Degree centrality
degree_centrality_after = nx.centrality.degree_centrality(G2_copy)
sort_degree_centrality_after = sorted(degree_centrality_after.items(), key=lambda item: item[1], reverse=True)
print("Done with degree centrality")

# Betweenness

betweenness_after = nx.betweenness_centrality(G2_copy)
sort_betweenness_after = sorted(betweenness_after.items(), key=lambda item: item[1], reverse=True)
print("Done with betweenness centrality")

# Closeness

closeness_after = nx.closeness_centrality(G2_copy)
sort_closeness_after = sorted(closeness_after.items(), key=lambda item: item[1], reverse=True)
print("Done with closeness centrality")


# Influence maximization
# Pagerank

pagerank_after = nx.pagerank(G2_copy)
sort_pagerank_after = sorted(pagerank_after.items(), key=lambda item: item[1], reverse=True)
print("Done with pagerank centrality")

# Eigenvector

eigenvector_after = nx.eigenvector_centrality(G2_copy)
sort_eigenvector_after = sorted(eigenvector_after.items(), key=lambda item: item[1], reverse=True)
print("Done with eigenvector centrality")

In [ ]:
# To only get the keys from the sorted list

only_keys_after = [k for k,v in sort_pagerank_after]
print(only_keys_after)

# Take the top 50 nodes to be used as seeds later
first_100_after = only_keys_after[0:100]
print(first_100_after)

In [8]:
# Adding weight to the edges of the graph

# for u,v in G1.edges():
#     G1[u][v]['weight'] = 1/G1.in_degree(v)
# G1.get_edge_data(30,1412)
# G1.in_degree(1412)

# for u,v in G2.edges():
#     G2[u][v]['weight'] = 1/G2.degree(v)
# G2.get_edge_data(0,5)

degree_dict = dict(G2_copy.degree())
weights = []
for u, v, data in G2_copy.edges(data=True):
    ratio = 1 / degree_dict[v]
    data['weight'] = ratio
    weights.append(ratio)

In [ ]:
# Adds an attribute to the nodes of the graph, for example degree centrality

nx.set_node_attributes(G1, centrality, "centrality")
# nx.set_node_attributes(G2, centrality, "centrality")

In [ ]:
# Filtering the graph by removing edges based on their weight attribute

for u,v in list(G1.edges()):
    if G1[u][v]['weight'] < 0.04:
        G1.remove_edge(u,v)
G1_new = G1.copy()

# Remove nodes based on their degree

for u in list(G1.nodes()):
    if G1.degree(u)< 10:
        G1.remove_node(u)

# Remove nodes based on their degree centrality

for u,v in centrality.items():
    if v < 0.004:
        G1.remove_node(u)

In [ ]:
# Lists of sources and targets based on in/out degrees of nodes

sources = []
targets = []
for u in G1.nodes():
    if G1.in_degree(u) == 0:
        sources.append(u)
for v in G1.nodes():
    if G1.out_degree(v) == 0:
        targets.append(v)

In [ ]:
# Splitting the sources to smaller components (useful for seed selection)

sub_sources = sources[0:50]
print(sub_sources)


In [ ]:
import random
def random(nodes, n):
    ''' Select seeds randomly
    Args:
        nodes (list) [#node]: node list of the graph;
        n (int): number of seeds;
    Returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    # random.sample(nodes, n)
    # return nodes[:n]
    # np.random.shuffle(nodes)
    # return nodes[:n]
    seeds = random.sample(nodes, n)
    return seeds

In [ ]:
def mia(nodes, edges, n):
    ''' select seeds by mia policy
    args:
        nodes (list) [#node]: node list of the graph;
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_connection = {}
    in_connection = {}
    centrality_score = {}
    seeds = []
    for edge in edges:
        if edge[0] in out_connection:
            out_connection[edge[0]].append(edge[1])
        else:
            out_connection[edge[0]] = [edge[1]]
        if edge[1] in in_connection:
            in_connection[edge[1]] += 1
        else:
            in_connection[edge[1]] = 1
    for node in nodes:
        centrality_score[node] = mia_centrality(node, out_connection, in_connection)
    i = 0
    for node, _ in sorted(centrality_score.items(), key=lambda item: item[1], reverse=True):
        if i >= n:
            break
        else:
            i += 1
        seeds.append(node)
    return seeds

def mia_centrality(node, out_connection, in_connection):
    ''' select seeds by mia centrality policy
    '''
    theta = 0.5
    c_score = 0
    q = 1  # threshold of influence
    visited = set()
    path_prob = 1
    c_score = dfs(visited, out_connection, path_prob, in_connection, node, theta, q)
    return c_score

def dfs(visited, out_connection, path_prob, in_connection, node, theta, q):
    if node not in visited:
        visited.add(node)
        if node in out_connection:
            for neighbour in out_connection[node]:
                path_prob *= q / in_connection[neighbour]
                if path_prob >= theta:
                    dfs(visited, out_connection, path_prob, in_connection, neighbour, theta, q)
                    path_prob /= (q / in_connection[neighbour])
                else:
                    path_prob /= (q / in_connection[neighbour])
    N_of_nodes = len(visited)
    return N_of_nodes

In [ ]:
def degree(edges, n):
    ''' Select seeds by degree policy
    Args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    Returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    degree = {}
    seeds = []
    for edge in edges:
        if edge[0] in degree:
            degree[edge[0]] += 1
        else:
            degree[edge[0]] = 1
        if edge[1] in degree:
            degree[edge[1]] += 1
        else:
            degree[edge[1]] = 1
    seeds = list({k: v for k, v in sorted(degree.items(), key=lambda item: item[1], reverse=True)}.keys())[:n]
    return seeds

In [ ]:
def degree_discount(edges, n):
    ''' Select seeds by degree discount degree
    Args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    Returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_degree = {}
    connection = {}
    seeds = []
    for edge in edges:
        if edge[1] in connection:
            connection[edge[1]].append(edge[0])
        else:
            connection[edge[1]] = [edge[0]]
        if edge[0] in out_degree:
            out_degree[edge[0]] += 1
        else:
            out_degree[edge[0]] = 1
    while len(seeds) < n:
        seed = sorted(out_degree.items(), key=lambda item: item[1], reverse=True)[0][0]
        seeds.append(seed)
        out_degree[seed] = -1
        if seed in connection:
            for node in connection[seed]:
                if node in out_degree:
                    out_degree[node] -= 1
    return seeds

In [ ]:
def degree_neighbor_fix(edges, n):
    ''' select seeds by degree neighbor fix policy
    args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_degree = {}
    centrality_score = {}
    connection = {}
    seeds = []
    for edge in edges:
        if edge[1] in connection:
            connection[edge[1]].append(edge[0])
        else:
            connection[edge[1]] = [edge[0]]
        if edge[0] in out_degree:
            out_degree[edge[0]] += 1
        else:
            out_degree[edge[0]] = 1
    centrality_score = out_degree.copy()
    for edge in edges:
        if edge[1] in out_degree:
            centrality_score[edge[0]] += out_degree[edge[1]]
    while len(seeds) < n:
        seed = sorted(centrality_score.items(), key=lambda item: item[1], reverse=True)[0][0]
        seeds.append(seed)
        centrality_score[seed] = -1
        if seed in connection:
            for node in connection[seed]:
                if node in out_degree:
                    centrality_score[node] -= out_degree[seed]

    return seeds

In [ ]:
def degree_neighbor(edges, n):
    ''' select seeds by degree neighbor policy
    args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_degree = {}
    centrality_score = {}
    seeds = []
    for edge in edges:
        if edge[0] in out_degree:
            out_degree[edge[0]] += 1
        else:
            out_degree[edge[0]] = 1
    centrality_score = out_degree.copy()
    for edge in edges:
        if edge[1] in out_degree:
            centrality_score[edge[0]] += out_degree[edge[1]]
    seeds = sorted(centrality_score.items(), key=lambda item: item[1], reverse=True)
    return seeds


In [ ]:
# Create communities based on the louvain method

louvain = nx.algorithms.community.louvain_communities(G2)
print(len(louvain))
# print(louvain[131])

In [ ]:
# Create a seed set based on the communities and a heuristic

seeds_new = {}
for community in louvain:
    max_node = max(community, key=lambda node: degree_centrality[node])
    seeds_new[max_node] = community

In [ ]:
# policy = 'degree'

# seeds_number = int(len(G1.nodes) * init_rate)
# seeds = list({k: v for k, v in sorted(dict(degreenew).items(), key=lambda item: item[1], reverse=True)}.keys())[:seeds_number]
policy = 'mia'

init_rate = 0.02
# nodes = G2_new.nodes
# edges = G2_new.edges
# # nodes = G_new.vs
# # edges = G_new.es
nodes = G2.nodes()
edges = G2.edges()
seeds_number = int(len(nodes) * init_rate)

if policy == 'degree':
    seeds = degree(edges, seeds_number)
    # seeds = list({k: v for k, v in sorted(dict(new_degree).items(), key=lambda item: item[1], reverse=True)}.keys())[:seeds_number]
if policy == 'random':
    seeds = random(nodes, seeds_number)
if policy == 'degree_discount':
    seeds = degree_discount(edges, seeds_number)
if policy == 'mia':
   seeds = mia(nodes, edges, seeds_number) 
if policy == 'degree_neighbor_fix':
    seeds = degree_neighbor_fix(edges, seeds_number)
if policy == 'degree_neighbor':
    seeds = degree_neighbor(edges, seeds_number)
print(f'Number of seeds: {len(seeds)}')

In [ ]:
print(seeds)

Use louvain communities without fairness measure and different centrality measures

In [ ]:
import community

# Partition into communities
partition = community.best_partition(G2)
for node in G2.nodes:
    G2.nodes[node]['community'] = partition[node]
print(len(partition))

num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

In [ ]:
# Calculate centralities (can also be done from the cell above, have to check to decide), for fairness this cell can be used to removed the selected nodes

seeds = set()
for community_id in set(partition.values()):
    community_nodes = [node for node in G2.nodes if G2.nodes[node]['community'] == community_id]
    max_degree_node = max(community_nodes, key=lambda node: pagerank[node])
    seeds.add(max_degree_node)
print("Number of seeds:", len(seeds))

In [ ]:
# Create a copy of the original graph to be modified later, by removing nodes
G2_copy_comm = G2.copy()


G2_copy_comm.remove_nodes_from(seeds)
print(G2_copy_comm)

In [ ]:
# Centrality measures

# Degree
# sort_degree = sorted(dict(G2.degree).items(), key=lambda item: item[1], reverse=True)
# print(sort_degree)

# Degree centrality
degree_centrality_after = nx.centrality.degree_centrality(G2_copy_comm)
sort_degree_centrality_after = sorted(degree_centrality_after.items(), key=lambda item: item[1], reverse=True)
print("Degree centrality done")

# Betweenness

betweenness_after = nx.betweenness_centrality(G2_copy_comm)
sort_betweenness_after = sorted(betweenness_after.items(), key=lambda item: item[1], reverse=True)
print("Betweenness centrality done")

# Closeness

closeness_after = nx.closeness_centrality(G2_copy_comm)
sort_closeness_after = sorted(closeness_after.items(), key=lambda item: item[1], reverse=True)
print("Closeness centrality done")


# Influence maximization
# Pagerank

pagerank_after = nx.pagerank(G2_copy_comm)
sort_pagerank_after = sorted(pagerank_after.items(), key=lambda item: item[1], reverse=True)
print("Pagerank done")

# Eigenvector

eigenvector_after = nx.eigenvector_centrality(G2_copy_comm)
sort_eigenvector_after = sorted(eigenvector_after.items(), key=lambda item: item[1], reverse=True)
print("Eigenvector done")

In [ ]:
import community

# Partition into communities
partition = community.best_partition(G2_copy_comm)
for node in G2_copy_comm.nodes:
    G2_copy_comm.nodes[node]['community'] = partition[node]
print(len(partition))

num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

In [ ]:
# Calculate centralities (can also be done from the cell above, have to check to decide)

seeds_after = set()
for community_id in set(partition.values()):
    community_nodes = [node for node in G2_copy_comm.nodes if G2_copy_comm.nodes[node]['community'] == community_id]
    max_degree_node = max(community_nodes, key=lambda node: pagerank_after[node])
    seeds_after.add(max_degree_node)
print("Number of seeds:", len(seeds_after))

The "correct" implementation for the equality fairness is bellow (as implemented in Farnandi's paper)

In [4]:
import community
# Calculcate communities using louvain method
partition_original = community.best_partition(G2)
for node in G2.nodes:
    G2.nodes[node]['community'] = partition_original[node]
print(len(partition_original))

num_distinct_values = len(set(partition_original.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

7624
The dictionary has 29 distinct values.


In [5]:
# Create a dictionary to store the population ratio of each community:

community_sizes = {}
for c in set(partition_original.values()):
    nodes_in_community = [n for n in partition_original.keys() if partition_original[n] == c]
    community_sizes[c] = len(nodes_in_community)

# Calculate the centralities for each node in the graph

# closeness_centrality = nx.closeness_centrality(G2)
# betweenness_centrality = nx.betweenness_centrality(G2)
# degree_centrality = nx.degree_centrality(G2)
# eigen_centrality = nx.eigenvector_centrality(G2)
pagerank_centrality = nx.pagerank(G2)

# Create a dictionary to store the centralities for each community:

# community_closeness = {}
# community_betweenness = {}
# community_degree = {}
# community_eigen = {}
community_pagerank = {}
for node in G2.nodes():
    community = partition_original[node]
    # if community not in community_closeness:
    #     community_closeness[community] = 0
    # if community not in community_betweenness:
    #     community_betweenness[community] = 0
    # if community not in community_degree:
    #     community_degree[community] = 0
    # if community not in community_eigen:
    #     community_eigen[community] = 0
    if community not in community_pagerank:
        community_pagerank[community] = 0

    # community_closeness[community] += closeness_centrality[node]
    # community_betweenness[community] += betweenness_centrality[node]
    # community_degree[community] += degree_centrality[node]
    # community_eigen[community] += eigen_centrality[node]
    community_pagerank[community] += pagerank_centrality[node]


# Calculate the total centrality for each community

# for community in community_closeness:
#     community_closeness[community] /= community_sizes[community]
# for community in community_betweenness:
#     community_betweenness[community] /= community_sizes[community]
# for community in community_degree:
#     community_degree[community] /= community_sizes[community]
# for community in community_eigen:
#     community_eigen[community] /= community_sizes[community]
for community in community_pagerank:
    community_pagerank[community] /= community_sizes[community]

# Calculate the number of seeds for each community proportional to the population ratio using the centralities:
seeds = {}
for node in G2.nodes():
    community = partition_original[node]
    if community not in seeds:
        seeds[community] = []
        # if len(seeds[community]) < community_sizes[community] * community_closeness[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_betweenness[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_degree[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_eigen[community]:
        #     seeds[community].append(node)
        if len(seeds[community]) < community_sizes[community] * community_pagerank[community]:
            seeds[community].append(node)

print("Number of seeds:", len(seeds))

Number of seeds: 29


In [8]:
# Create a copy of the original graph to be modified later, by removing nodes (fair version)
G2_copy_comm_fair = G2.copy()


G2_copy_comm_fair.remove_nodes_from(seeds)
print(G2_copy_comm_fair)

Graph with 7596 nodes and 27593 edges


In [ ]:
degree_dict = dict(G2_copy_comm_fair.degree())
weights = []
for u, v, data in G2_copy_comm_fair.edges(data=True):
    ratio = 1 / degree_dict[v]
    data['weight'] = ratio
    weights.append(ratio)

print(weights)

In [10]:
import community
# Calculcate communities using louvain method
partition = community.best_partition(G2_copy_comm_fair)
for node in G2_copy_comm_fair.nodes:
    G2_copy_comm_fair.nodes[node]['community'] = partition[node]
print(len(partition))

num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

7596
The dictionary has 62 distinct values.


In [11]:
community_sizes = {}
for c in set(partition.values()):
    nodes_in_community = [n for n in partition.keys() if partition[n] == c]
    community_sizes[c] = len(nodes_in_community)

# Calculate the centralities for each node in the graph

closeness_centrality = nx.closeness_centrality(G2_copy_comm_fair)
betweenness_centrality = nx.betweenness_centrality(G2_copy_comm_fair)
degree_centrality = nx.degree_centrality(G2_copy_comm_fair)
eigen_centrality = nx.eigenvector_centrality(G2_copy_comm_fair)
pagerank_centrality = nx.pagerank(G2_copy_comm_fair)

In [24]:
# Create a dictionary to store the population ratio of each community:

# community_sizes = {}
# for c in set(partition.values()):
#     nodes_in_community = [n for n in partition.keys() if partition[n] == c]
#     community_sizes[c] = len(nodes_in_community)

# # Calculate the centralities for each node in the graph

# # closeness_centrality = nx.closeness_centrality(G2_copy_comm_fair)
# # betweenness_centrality = nx.betweenness_centrality(G2_copy_comm_fair)
# degree_centrality = nx.degree_centrality(G2_copy_comm_fair)
# # eigen_centrality = nx.eigenvector_centrality(G2_copy_comm_fair)
# # pagerank_centrality = nx.pagerank(G2_copy_comm_fair)

# Create a dictionary to store the centralities for each community:

community_closeness = {}
community_betweenness = {}
community_degree = {}
community_eigen = {}
community_pagerank = {}
for node in G2_copy_comm_fair.nodes():
    community = partition[node]
    if community not in community_closeness:
        community_closeness[community] = 0
    # if community not in community_betweenness:
    #     community_betweenness[community] = 0
    # if community not in community_degree:
    #     community_degree[community] = 0
    # if community not in community_eigen:
    #     community_eigen[community] = 0
    # if community not in community_pagerank:
    #     community_pagerank[community] = 0

    community_closeness[community] += closeness_centrality[node]
    # community_betweenness[community] += betweenness_centrality[node]
    # community_degree[community] += degree_centrality[node]
    # community_eigen[community] += eigen_centrality[node]
    # community_pagerank[community] += pagerank_centrality[node]


# Calculate the total centrality for each community

for community in community_closeness:
    community_closeness[community] /= community_sizes[community]
# for community in community_betweenness:
#     community_betweenness[community] /= community_sizes[community]
# for community in community_degree:
#     community_degree[community] /= community_sizes[community]
# for community in community_eigen:
#     community_eigen[community] /= community_sizes[community]
# for community in community_pagerank:
#     community_pagerank[community] /= community_sizes[community]

# Calculate the number of seeds for each community proportional to the population ratio using the centralities:
seeds = {}
for node in G2_copy_comm_fair.nodes():
    community = partition[node]
    if community not in seeds:
        seeds[community] = []
        if len(seeds[community]) < community_sizes[community] * community_closeness[community]:
            seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_betweenness[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_degree[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_eigen[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_pagerank[community]:
        #     seeds[community].append(node)

print("Number of seeds:", len(seeds))

Number of seeds: 62


The new correct implementation for fairness

In [4]:
import community

# Compute the Louvain communities
partition = community.best_partition(G2)

# Compute the size of each community
community_sizes = np.bincount(list(partition.values()))

# Compute the total number of nodes in the graph
total_nodes = len(G2.nodes())

# Compute the number of communities and print it
num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

proportional_sizes = community_sizes / total_nodes

The dictionary has 29 distinct values.


In [5]:
# Initialize a dictionary to store the seed nodes
seeds = {}
for community_id in set(partition.values()):
    nodes_in_community = [node for node, comm in partition.items() if comm == community_id]

    # Compute the number of seed nodes for the current community
    num_seeds = int(round(proportional_sizes[community_id] * len(nodes_in_community)))   

    # Sort the nodes in the current community by centrality measure
    # nodes_sorted_by_dc = sorted(nodes_in_community, key=lambda node: degree_centrality[node], reverse=True)
    nodes_sorted_by_bc = sorted(nodes_in_community, key=lambda node: betweenness[node], reverse=True)
    # nodes_sorted_by_cc = sorted(nodes_in_community, key=lambda node: closeness[node], reverse=True)
    # nodes_sorted_by_pc = sorted(nodes_in_community, key=lambda node: pagerank[node], reverse=True)
    # nodes_sorted_by_ec = sorted(nodes_in_community, key=lambda node: eigenvector[node], reverse=True)
    
    # Add the top `num_seeds` nodes to the seed dictionary 
    # seeds.update({node: degree_centrality[node] for node in nodes_sorted_by_dc[:num_seeds]})
    seeds.update({node: betweenness[node] for node in nodes_sorted_by_bc[:num_seeds]})
    # seeds.update({node: closeness[node] for node in nodes_sorted_by_cc[:num_seeds]})
    # seeds.update({node: pagerank[node] for node in nodes_sorted_by_pc[:num_seeds]})
    # seeds.update({node: eigenvector[node] for node in nodes_sorted_by_ec[:num_seeds]})

print(len(seeds))


1563


In [29]:
print(seeds)

{11: 0.44881580653196423, 789: 0.42554206110590714, 311: 0.4209970900944556, 72: 0.4207951003666905, 600: 0.41749003353053654, 1055: 0.41565980471055675, 36: 0.4148978477626514, 173: 0.414554653087988, 87: 0.4133335789993636, 1297: 0.4111536773228361, 762: 0.4111054961923125, 784: 0.4088774768947667, 127: 0.40439985611032836, 319: 0.40435324478528106, 407: 0.4043066442038837, 306: 0.4028210761002524, 634: 0.4014382368414119, 1167: 0.4007503714595726, 310: 0.3997002109783638, 663: 0.39777410870878155, 432: 0.39772901215997947, 1661: 0.39716616684672235, 86: 0.39671703627243, 290: 0.39669460640663745, 1026: 0.3956655672201092, 47: 0.3936675926339111, 695: 0.3928520438697759, 6: 0.3923467820563774, 697: 0.3918428182460738, 20: 0.39103480485505193, 56: 0.3895800945871292, 166: 0.38947196799967787, 769: 0.3892127089096465, 29: 0.3880932299084129, 1286: 0.38768579420456384, 71: 0.38738612541487394, 3755: 0.38727921308793933, 371: 0.38725783770362054, 1471: 0.38700151701677865, 1185: 0.386426

In [6]:
# Create a copy of the original graph to be modified later, by removing nodes (fair version)
G2_copy_comm_fair = G2.copy()


G2_copy_comm_fair.remove_nodes_from(seeds)
print(G2_copy_comm_fair)

Graph with 5552 nodes and 5974 edges


In [7]:
# Calcualte the centralities for the new graph

degree_after = nx.degree_centrality(G2_copy_comm_fair)
closeness_after = nx.closeness_centrality(G2_copy_comm_fair)
betweenness_after = nx.betweenness_centrality(G2_copy_comm_fair)
degree_after = nx.degree_centrality(G2_copy_comm_fair)
eigen_after = nx.eigenvector_centrality(G2_copy_comm_fair)
pagerank_after = nx.pagerank(G2_copy_comm_fair)

In [8]:
import community

# Compute the Louvain communities
partition_after = community.best_partition(G2_copy_comm_fair)

# Compute the size of each community

community_sizes_after = np.bincount(list(partition_after.values()))

# Compute the total number of nodes in the graph
total_nodes_after = len(G2_copy_comm_fair.nodes())

# Compute the number of communities and print it
num_distinct_values = len(set(partition_after.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

proportional_sizes_after = community_sizes_after / total_nodes_after

The dictionary has 3636 distinct values.


In [57]:
print(type(partition_after))

<class 'dict'>


In [50]:
# Initialize a dictionary to store the seed nodes
seeds_after = {}
for community_id in set(partition_after.values()):
    nodes_in_community_after = [node for node, comm in partition_after.items() if comm == community_id]

    # Compute the number of seed nodes for the current community
    num_seeds_after = int(round(proportional_sizes_after[community_id] * len(nodes_in_community_after)))   

    # Sort the nodes in the current community by betweenness centrality
    # nodes_sorted_by_dc_after = sorted(nodes_in_community_after, key=lambda node: degree_after[node], reverse=True)
    # nodes_sorted_by_bc_after = sorted(nodes_in_community_after, key=lambda node: betweenness_after[node], reverse=True)
    # nodes_sorted_by_cc_after = sorted(nodes_in_community_after, key=lambda node: closeness_after[node], reverse=True)
    nodes_sorted_by_pc_after = sorted(nodes_in_community_after, key=lambda node: pagerank_after[node], reverse=True)
    # nodes_sorted_by_ec_after = sorted(nodes_in_community_after, key=lambda node: eigen_after[node], reverse=True)
    
    # Add the top `num_seeds` nodes to the seed dictionary 
    # seeds_after.update({node: degree_after[node] for node in nodes_sorted_by_dc_after[:num_seeds_after]})
    # seeds_after.update({node: betweenness_after[node] for node in nodes_sorted_by_bc_after[:num_seeds_after]})
    # seeds_after.update({node: closeness_after[node] for node in nodes_sorted_by_cc_after[:num_seeds_after]})
    seeds_after.update({node: pagerank_after[node] for node in nodes_sorted_by_pc_after[:num_seeds_after]})
    # seeds_after.update({node: eigen_after[node] for node in nodes_sorted_by_ec_after[:num_seeds_after]})

print(len(seeds_after))

128


In [ ]:
print(len(seeds_after))

For the following code dont forget to change the graphs for before/after blocking the nodes

In [51]:
# ICM
# This code can be used with the previous two cell blocks to calculate the influence count. 
# It combines the method of calculating the spread using spreading model as defined in the NDlib package, witht the different heuristic approaches.
average = []
for index in range(7):
    model = ep.IndependentCascadesModel(G2_copy_comm_fair)
    # model = ep.SIRModel(G2)
    # model = ep.SIModel(G2)
    # model = ep.ThresholdModel(G2_copy_comm_fair)
    cfg = mc.Configuration()
    # cfg.add_model_parameter('beta', 0.01)
    # cfg.add_model_parameter('gamma', 0.005)
    # cfg.add_model_parameter('lambda', 0.005)
    infected_nodes = seeds_after
    cfg.add_model_initial_configuration("Infected", infected_nodes)
    # cfg.add_model_parameter("fraction_infected", 0.05)
    model.set_initial_status(cfg)
    threshold = 0.1
    for e in G2_copy_comm_fair.edges():
        cfg.add_edge_configuration("threshold", e, random.uniform(0,1))
    # for n in G2.nodes():
    #     cfg.add_node_configuration("threshold", n, threshold)

    # iterations = model.iteration_bunch(500, True, True)
    # trends = model.build_trends(iterations)
    # viz = DiffusionTrend(model, trends)
    # p = viz.plot(width=800, height=800)
    # show(p)
    list = []
    for i in range(0,1000):
        iterations = model.iteration_bunch(1)
        # print(iterations)

        dict = iterations[0]
        for i in dict:
            k = (i,dict[i])
            list.append(k)
    list2 = []
    for i in range(len(list)):
        if list[i][0] == 'status':
            list2.append(list[i][1])
    list4 = []
    for t in range(len(list2)):
        for f in list2[t]:
            if list2[t][f] == 1:
                list4.append(f)
    # print("================================")
    # print(f"Results for iteration {index+1}:")
    # print("Total nodes infected: ", len(list4), "out of", len(G2.nodes()))
    percentage = (len(list4)/len(G2_copy_comm_fair.nodes()))*100
    # print("Percentage of total infected nodes with current seed set: ", percentage, "%")
    # print("================================")
    average.append(percentage)
print("The average percentage is: ", sum(average)/len(average), "%")

The average percentage is:  7.081850533807829 %


In [52]:
# LTM
# This code can be used with the previous two cell blocks to calculate the influence count. 
# It combines the method of calculating the spread using spreading model as defined in the NDlib package, witht the different heuristic approaches.
average = []
for index in range(7):
    # model = ep.IndependentCascadesModel(G2)
    # model = ep.SIRModel(G2)
    # model = ep.SIModel(G2)
    model = ep.ThresholdModel(G2_copy_comm_fair)
    cfg = mc.Configuration()
    # cfg.add_model_parameter('beta', 0.01)
    # cfg.add_model_parameter('gamma', 0.005)
    # cfg.add_model_parameter('lambda', 0.005)
    infected_nodes = seeds_after
    cfg.add_model_initial_configuration("Infected", infected_nodes)
    # cfg.add_model_parameter("fraction_infected", 0.05)
    model.set_initial_status(cfg)
    threshold = 0.1
    # for e in G2.edges():
    #     cfg.add_edge_configuration("threshold", e, threshold)
    for n in G2_copy_comm_fair.nodes():
        cfg.add_node_configuration("threshold", n, random.uniform(0,1))

    # iterations = model.iteration_bunch(500, True, True)
    # trends = model.build_trends(iterations)
    # viz = DiffusionTrend(model, trends)
    # p = viz.plot(width=800, height=800)
    # show(p)
    list = []
    for i in range(0,1000):
        iterations = model.iteration_bunch(1)

        dict = iterations[0]
        for i in dict:
            k = (i,dict[i])
            list.append(k)
    list2 = []
    for i in range(len(list)):
        if list[i][0] == 'status':
            list2.append(list[i][1])
    list4 = []
    for t in range(len(list2)):
        for f in list2[t]:
            if list2[t][f] == 1:
                list4.append(f)
    # print("================================")
    # print(f"Results for iteration {index+1}:")
    # print("Total nodes infected: ", len(list4), "out of", len(G2.nodes()))
    percentage = (len(list4)/len(G2_copy_comm_fair.nodes()))*100
    # print("Percentage of total infected nodes with current seed set: ", percentage, "%")
    # print("================================")
    average.append(percentage)
print("The average percentage is: ", sum(average)/len(average), "%")

The average percentage is:  23.820538891713266 %


In [ ]:
# ICM
# This code can be used with the previous two cell blocks to calculate the influence count. 
# It combines the method of calculating the spread using spreading model as defined in the NDlib package, witht the different heuristic approaches.
average = []
for index in range(7):
    model = ep.IndependentCascadesModel(G2_copy)
    # model = ep.SIRModel(G2)
    # model = ep.SIModel(G2)
    # model = ep.ThresholdModel(G2_copy_comm_fair)
    cfg = mc.Configuration()
    # cfg.add_model_parameter('beta', 0.01)
    # cfg.add_model_parameter('gamma', 0.005)
    # cfg.add_model_parameter('lambda', 0.005)
    infected_nodes = first_100_after
    cfg.add_model_initial_configuration("Infected", infected_nodes)
    # cfg.add_model_parameter("fraction_infected", 0.05)
    model.set_initial_status(cfg)
    threshold = 0.1
    for e in G2_copy.edges():
        cfg.add_edge_configuration("threshold", e, 0.1)
    # for n in G2.nodes():
    #     cfg.add_node_configuration("threshold", n, threshold)

    # iterations = model.iteration_bunch(500, True, True)
    # trends = model.build_trends(iterations)
    # viz = DiffusionTrend(model, trends)
    # p = viz.plot(width=800, height=800)
    # show(p)
    list = []
    for i in range(0,50):
        iterations = model.iteration_bunch(200)

        dict = iterations[0]
        for i in dict:
            k = (i,dict[i])
            list.append(k)
    list2 = []
    for i in range(len(list)):
        if list[i][0] == 'status':
            list2.append(list[i][1])
    list4 = []
    for t in range(len(list2)):
        for f in list2[t]:
            if list2[t][f] == 1:
                list4.append(f)
    # print("================================")
    # print(f"Results for iteration {index+1}:")
    # print("Total nodes infected: ", len(list4), "out of", len(G2.nodes()))
    percentage = (len(list4)/len(G2_copy.nodes()))*100
    # print("Percentage of total infected nodes with current seed set: ", percentage, "%")
    # print("================================")
    average.append(percentage)
print("The average percentage is: ", sum(average)/len(average), "%")